# Initialization of libraries

In [ ]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Our predefined dataset with questions we will load into our models

In [ ]:
pr_qst = pd.DataFrame([ ## dataset
    {"question": "What is the historical significance of the mosaic in the apse of the Church of Panagia Aggeloktisti and in what century was it created?",
     "model_response": "",
     },
    {"question": "What architectural features distinguish the Church of Panagia Aggeloktisti and what periods of construction does it unite?",
     "model_response": "",
     },
    {"question": "What does the name Aggeloktisti mean and where does it come from?",
     "model_response": "",
     },
    {"question": "What chapels were added to the church in the Middle Ages and what was their purpose?",
     "model_response": "",
     },
    {"question": "How does the Church of Panagia Aggeloktisti demonstrate the cultural and artistic links between different regions of the Mediterranean?",
     "model_response": "",
     },
    {"question": "What arguments are put forward for the inclusion of the Church of Panagia Aggeloktisti in the UNESCO World Heritage List under criterion (i)?",
     "model_response": "",
     },
    {"question": "What conservation measures have been taken since 1952 to preserve the mosaics and other elements of the church?",
     "model_response": "",
     },
    {"question": "How does the church reflect the religious and spiritual life of the local community over the centuries?",
     "model_response": "",
    },
    {"question": "How does the church of Panagia Aggeloktisti maintain its authenticity and integrity despite the passing centuries and the restorations carried out?",
     "model_response": "",
    }
])

# Models init

In [ ]:
## llama

# llama = "openlm-research/open_llama_3b"
# llama_model = AutoModelForCausalLM.from_pretrained(
#     llama,
#     torch_dtype=torch.float16, ## gpu support
#     device_map="auto",
#     )
# llama_tokenizer = AutoTokenizer.from_pretrained(llama)


# tiny-llama
tiny = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tiny_model = AutoModelForCausalLM.from_pretrained(
    tiny,
    torch_dtype=torch.float16, ## gpu support
    device_map="auto",
    )
tiny_tokenizer = AutoTokenizer.from_pretrained(tiny)


## phi
# phi = "microsoft/phi-2"
# phi_model = AutoModelForCausalLM.from_pretrained(
#     phi,
#     torch_dtype=torch.float16, ## gpu support
#     device_map="auto",
#     trust_remote_code=True ## fix api issues
#     )
# phi_tokenizer = AutoTokenizer.from_pretrained(phi)

# falcon
# falcon = "tiiuae/falcon-rw-1b"
# falcon_model = AutoModelForCausalLM.from_pretrained(
#     falcon,
#     torch_dtype=torch.float16, ## gpu support
#     device_map="auto",
#     offload_folder="offload" ## memory issue
#     )
# falcon_tokenizer = AutoTokenizer.from_pretrained(falcon)


# Generate function to talk with LLM, responses to store output, output function to store in a list, after that save it in .csv file

In [ ]:
def gen(question, model, tokenizer):
    prompt = f"Question: {question}\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_length=100,
        num_return_sequences=1,
        do_sample=False, ## variety, turn off for now
        top_p=0.95,
        temperature=0.7, ## temp
        pad_token_id=tokenizer.eos_token_id,
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = answer.split("Answer:")[-1].strip()
    return answer

# outputs
llama_responses = [] # 1 minute 17 seconds runtime (3b model)
falcon_responses = [] # 45 seconds (1b model)
phi_responses = [] # 20 seconds -> 100 max_length (2b? model)
tiny_responses = [] # 17 seconds -> 100 max_length (1b model)

# dataframes
llama_dataframe = pd.DataFrame()
falcon_dataframe = pd.DataFrame()
phi_dataframe = pd.DataFrame()
tiny_dataframe = pd.DataFrame()

def output(model, tokenizer, responses, dataframe):
    for question in pr_qst["question"]:
        resp = gen(question, model, tokenizer)
        responses.append(resp)
    dataframe = pr_qst.copy()
    dataframe["model_response"] = responses
    return dataframe

## llama
# llama_result = output(llama_model, llama_tokenizer, llama_responses, llama_dataframe)
# llama_result.to_csv("llama_results.csv")

## falcon
# falcon_result = output(falcon_model, falcon_tokenizer, falcon_responses, falcon_dataframe)
# falcon_result.to_csv("falcon_results.csv")

## phi
# phi_result = output(phi_model, phi_tokenizer, phi_responses, phi_dataframe)
# phi_result.to_csv("phi_results.csv")

## tiny llama
result = output(tiny_model, tiny_tokenizer, tiny_responses, tiny_dataframe)
result.to_csv("tiny_llama_results.csv")
